This scripts matches the naming conventions of IPYS data collection with the naming and codes from our geographic files. IPYS names were taken from [this spreadheet](https://docs.google.com/spreadsheets/d/1EbW1r-uiWH_2Vc2WZSUjFpgFHr_2U0OzvgdKnNhtWZw/edit?usp=sharing).

In [1]:
from fuzzymatcher import link_table, fuzzy_left_join
from fuzzywuzzy import fuzz, process
import geopandas as gpd
import pandas as pd

In [2]:
# Reads both files
ipys_names = pd.read_csv("../input/code-correspondence.csv")
geo_names = gpd.read_file("../output/level-2.geojson")

In [3]:
ipys_names.head()

,level_1_name,level_2_name,level_1_code,level_2_code
0,Amazonas,Alto Orinoco (La Esmeralda),NaN,NaN
1,Amazonas,Atabapo (San Fernando de Atabapo),NaN,NaN
2,Amazonas,Atures (Puerto Ayacucho),NaN,NaN
3,Amazonas,Autana (Isla Ratón),NaN,NaN
4,Amazonas,Manapiare (San Juan de Manapiare),NaN,NaN


In [4]:
geo_names.head()

,parent_code,parent_name,code,name,geometry
0,VE02,Amazonas,VE0202,Autónomo Atabapo,"POLYGON ((-66.33494 4.26540, -66.27128 4.22420..."
1,VE02,Amazonas,VE0207,Autónomo Rio Negro,"POLYGON ((-65.92352 2.88024, -65.92278 2.87806..."
2,VE02,Amazonas,VE0205,Autónomo Maroa,"POLYGON ((-67.43485 3.24847, -67.43485 3.24847..."
3,VE02,Amazonas,VE0204,Autónomo Autana,"POLYGON ((-66.41415 5.73555, -66.41180 5.72659..."
4,VE02,Amazonas,VE0203,Autónomo Atures,"POLYGON ((-67.43503 6.18680, -67.43404 6.18646..."


In [5]:
# Keeps only needed columns
geo_names = geo_names[['parent_name', 'parent_code', 'name', 'code']]

In [6]:
# Changes 'La Guaira' for 'Vargas' and standardizes Distrito Capital spelling
geo_names['parent_name'] = geo_names.parent_name.replace({'La Guaira': 'Vargas', 'Distrito Capital': 'Distrito capital'})

In [7]:
# Makes everything else lowercase
# ipys_names = ipys_names.apply(lambda x: x.str.lower(), axis=1)
# level_2 = level_2.apply(lambda x: x.str.lower(), axis=1)

In [8]:
geo_names

,parent_name,parent_code,name,code
0,Amazonas,VE02,Autónomo Atabapo,VE0202
1,Amazonas,VE02,Autónomo Rio Negro,VE0207
2,Amazonas,VE02,Autónomo Maroa,VE0205
3,Amazonas,VE02,Autónomo Autana,VE0204
4,Amazonas,VE02,Autónomo Atures,VE0203
...,...,...,...,...
361,Zulia,VE23,Cabimas,VE2303
362,Zulia,VE23,Lagunillas,VE2310
363,Zulia,VE23,Valmore Rodríguez,VE2321
364,Zulia,VE23,Mara,VE2312


In [9]:
# Let's try a quick and dirty fuzzy matcher
# https://stackoverflow.com/a/56315491

def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1

In [10]:
# To improve accuracy, let's do it within each state and get the closet match.
# As we do it, let's also display the results so we can manually correct them later
new_df = []
for level_1_name in ipys_names.level_1_name.unique():
    
    subset_ipys = ipys_names[ipys_names.level_1_name==level_1_name].reset_index()
    subset_geo = geo_names[geo_names.parent_name==level_1_name].reset_index()
    
    #display(subset_ipys, subset_geo)
    
    # We will remove the parts of the name within the parenthesis of IPYS naming for easy matching
    subset_ipys['level_2_name_simplified'] = subset_ipys['level_2_name'].str.replace(r"\(.*\)","")
    
    merge = fuzzy_left_join(subset_ipys, subset_geo, 'level_2_name_simplified', 'name')
    #merge = fuzzy_merge(subset_ipys, subset_geo, 'level_2_name', 'name')
        
    # Renaming and filtering columns for clarity
    merge = merge[['level_1_name', 'level_2_name','parent_name', 'name',  'parent_code', 'code']]
    
    merge = merge.rename(columns={
        'level_1_name': 'IPYS_level_1_name',
        'level_2_name': 'IPYS_level_2_name',
        'parent_name': 'geo_level_1_name',
        'parent_code': 'geo_level_1_code',
        'name': 'geo_level_2_name',
        'code': 'geo_level_2_code'
    })
    
    new_df.append(merge)
    
    display(merge)

/var/folders/52/vy6xx2q911l7rd368b7ngb8w0000gn/T/ipykernel_10399/855297660.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  subset_ipys['level_2_name_simplified'] = subset_ipys['level_2_name'].str.replace(r"\(.*\)","")


,IPYS_level_1_name,IPYS_level_2_name,geo_level_1_name,geo_level_2_name,geo_level_1_code,geo_level_2_code
0,Amazonas,Alto Orinoco (La Esmeralda),Amazonas,Autónomo Alto Orinoco,VE02,VE0201
1,Amazonas,Atabapo (San Fernando de Atabapo),Amazonas,Autónomo Atabapo,VE02,VE0202
2,Amazonas,Atures (Puerto Ayacucho),Amazonas,Autónomo Atures,VE02,VE0203
3,Amazonas,Autana (Isla Ratón),Amazonas,Autónomo Autana,VE02,VE0204
4,Amazonas,Manapiare (San Juan de Manapiare),Amazonas,Manapiare,VE02,VE0206
5,Amazonas,Maroa,Amazonas,Autónomo Maroa,VE02,VE0205
6,Amazonas,Río Negro (San Carlos de Río Negro),Amazonas,Autónomo Rio Negro,VE02,VE0207


/var/folders/52/vy6xx2q911l7rd368b7ngb8w0000gn/T/ipykernel_10399/855297660.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  subset_ipys['level_2_name_simplified'] = subset_ipys['level_2_name'].str.replace(r"\(.*\)","")


,IPYS_level_1_name,IPYS_level_2_name,geo_level_1_name,geo_level_2_name,geo_level_1_code,geo_level_2_code
0,Anzoátegui,Anaco,Anzoátegui,Anaco,VE03,VE0301
1,Anzoátegui,Aragua (Aragua de Barcelona),Anzoátegui,Aragua,VE03,VE0302
2,Anzoátegui,Bolívar (Barcelona),Anzoátegui,Simón Bolívar,VE03,VE0318
3,Anzoátegui,Bruzual (Clarines),Anzoátegui,Manuel Ezequiel Bruzual,VE03,VE0312
4,Anzoátegui,Cajigal (Onoto),Anzoátegui,Juan Manuel Cajigal,VE03,VE0309
5,Anzoátegui,Diego Bautista Urbaneja (Lechería),Anzoátegui,Diego Bautista Urbaneja,VE03,VE0321
6,Anzoátegui,Freites (Cantaura),Anzoátegui,Pedro María Freites,VE03,VE0313
7,Anzoátegui,Guanipa (San José de Guanipa),Anzoátegui,San José de Guanipa,VE03,VE0315
8,Anzoátegui,Guanta (Guanta),Anzoátegui,Guanta,VE03,VE0306
9,Anzoátegui,Independencia (Soledad),Anzoátegui,Independencia,VE03,VE0307


/var/folders/52/vy6xx2q911l7rd368b7ngb8w0000gn/T/ipykernel_10399/855297660.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  subset_ipys['level_2_name_simplified'] = subset_ipys['level_2_name'].str.replace(r"\(.*\)","")


...

,IPYS_level_1_name,IPYS_level_2_name,geo_level_1_name,geo_level_2_name,geo_level_1_code,geo_level_2_code
0,Vargas,Parroquia Caraballeda,Vargas,Caraballeda,VE24,VE240101
1,Vargas,Parroquia Carayaca,Vargas,Carayaca,VE24,VE240102
2,Vargas,Parroquia Carlos Soublette,Vargas,Carlos Soublette,VE24,VE240111
3,Vargas,Parroquia Caruao,Vargas,Caruao,VE24,VE240103
4,Vargas,Parroquia Catia La Mar,Vargas,Catia la Mar,VE24,VE240104
5,Vargas,Parroquia El Junko,Vargas,El Junko,VE24,VE240105
6,Vargas,Parroquia La Guaira,Vargas,La Guaira,VE24,VE240106
7,Vargas,Parroquia Macuto,Vargas,Macuto,VE24,VE240107
8,Vargas,Parroquia Maiquetía,Vargas,Maiquetía,VE24,VE240108
9,Vargas,Parroquia Naiguatá,NaN,NaN,NaN,NaN


/var/folders/52/vy6xx2q911l7rd368b7ngb8w0000gn/T/ipykernel_10399/855297660.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  subset_ipys['level_2_name_simplified'] = subset_ipys['level_2_name'].str.replace(r"\(.*\)","")


,IPYS_level_1_name,IPYS_level_2_name,geo_level_1_name,geo_level_2_name,geo_level_1_code,geo_level_2_code
0,Yaracuy,Arístides Bastidas (San Pablo),Yaracuy,Arístides Bastidas,VE22,VE2201
1,Yaracuy,Bolívar (Aroa),Yaracuy,Bolívar,VE22,VE2202
2,Yaracuy,Bruzual (Chivacoa),Yaracuy,Bruzual,VE22,VE2203
3,Yaracuy,Cocorote,Yaracuy,Cocorote,VE22,VE2204
4,Yaracuy,Independencia,Yaracuy,Independencia,VE22,VE2205
5,Yaracuy,La Trinidad (Boraure),Yaracuy,La Trinidad,VE22,VE2207
6,Yaracuy,Monge (Yumare),Yaracuy,Manuel Monge,VE22,VE2208
7,Yaracuy,Nirgua,Yaracuy,Nirgua,VE22,VE2209
8,Yaracuy,Páez (Sabana de Parra),Yaracuy,José Antonio Páez,VE22,VE2206
9,Yaracuy,Peña (Yaritagua),Yaracuy,Peña,VE22,VE2210


/var/folders/52/vy6xx2q911l7rd368b7ngb8w0000gn/T/ipykernel_10399/855297660.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  subset_ipys['level_2_name_simplified'] = subset_ipys['level_2_name'].str.replace(r"\(.*\)","")


,IPYS_level_1_name,IPYS_level_2_name,geo_level_1_name,geo_level_2_name,geo_level_1_code,geo_level_2_code
0,Zulia,Baralt (San Timoteo),Zulia,Baralt,VE23,VE2302
1,Zulia,Bolívar (Tía Juana),Zulia,Simón Bolívar,VE23,VE2319
2,Zulia,Cabimas,Zulia,Cabimas,VE23,VE2303
3,Zulia,Catatumbo (Encontrados),Zulia,Catatumbo,VE23,VE2304
4,Zulia,Colón (San Carlos del Zulia),NaN,NaN,NaN,NaN
5,Zulia,Guajira (Sinamaica),Zulia,Indígena Bolivariano Guajira,VE23,VE2315
6,Zulia,La Cañada de Urdaneta (Concepción),Zulia,La Cañada de Urdaneta,VE23,VE2309
7,Zulia,Lagunillas (Ciudad Ojeda),Zulia,Lagunillas,VE23,VE2310
8,Zulia,Lossada (La Concepción),Zulia,Jesús Enrique Lossada,VE23,VE2307
9,Zulia,Machiques,Zulia,Machiques de Perija,VE23,VE2311


In [11]:
new_df = pd.concat(new_df, ignore_index=True)

After processing, we will have the following entries to manually fix:
```
* Anzoátegui - McGregor (El Chaparro)
* Anzoátegui - Píritu
* Bolívar - Heres (Ciudad Bolívar)
* Cojedes - Falcón (Tinaquillo)
* Distrito Capital - Parroquia Antímano
* Falcón - Píritu (Falcón)
* Falcón - Tocópero
* Guárico - Ortíz 
* Lara - Morán (El Tocuyo)
* Mérida - Salas (Arapuey) 
* Portuguesa - Turén (Villa Bruzual)
* Táchira - Guásimos (Palmira) 
* Trujillo - Boconó
* Trujillo - Motatán
* Trujillo - Pampán
* Vargas - Parroquia Naiguatá
* Zulia - Colón (San Carlos del Zulia)
* Zulia - Semprún (Casigua El Cubo) 	
```

Notice that since all the erros are due to NOT MATCHING (as opposed to WRONG MATCHES), we can simply look at NaNs and we'll find them all.

In [12]:
new_df[new_df.geo_level_2_code.isna()]

,IPYS_level_1_name,IPYS_level_2_name,geo_level_1_name,geo_level_2_name,geo_level_1_code,geo_level_2_code
18,Anzoátegui,McGregor (El Chaparro),NaN,NaN,NaN,NaN
23,Anzoátegui,Píritu (Anzoátegui),NaN,NaN,NaN,NaN
69,Bolívar,Heres (Ciudad Bolívar),NaN,NaN,NaN,NaN
92,Cojedes,Falcón (Tinaquillo),NaN,NaN,NaN,NaN
105,Distrito capital,Parroquia Antímano,NaN,NaN,NaN,NaN
142,Falcón,Píritu (Falcón),NaN,NaN,NaN,NaN
146,Falcón,Tocópero,NaN,NaN,NaN,NaN
150,Guárico,Ortíz,NaN,NaN,NaN,NaN
169,Lara,Morán (El Tocuyo),NaN,NaN,NaN,NaN
193,Mérida,Salas (Arapuey),NaN,NaN,NaN,NaN


After manually inspecting the matches, those are the correspondences:

In [13]:
matches = {
	# On the left, the merged df index locator. On the right, the geographic file locator.
	18: 27,
	23: 8,
	69: 72, # I'm taking that Ciudad Bolívar, seat of the Heres municipality, is Angostura del Orinoco – as the website of https://www.alcaldiadeheres.gob.ve/ has a banner calling itself 'Angostura del Orinoco'
	92: 93,
	105: 107,
	142: 148,
	146: 137,
	150: 150,
	169: 179,
	193: 225, # Possible typo in geo_names, as the city is called Júlio Cesar Salas, with an extra S in the end
	255: 260,
	280: 325,
	301: 292,
	312: 300,
	313: 285,
    329: 171,
	349: 346,
	363: 359

}

In [14]:
for merged_loc, geo_loc in matches.items():
    
    
    for merged_column, geo_column in [
        ('geo_level_1_name', 'parent_name'), 
        ('geo_level_2_name',  'name'),
        ('geo_level_1_code', 'parent_code'),
        ('geo_level_2_code', 'code')
    ]:
        
        new_df.loc[merged_loc, merged_column] = geo_names.loc[geo_loc, geo_column]


Finally, saves to csv.

In [15]:
new_df.to_csv("../output/correspondence.csv", index=False, sep=";")